# Filter inappropriate content

- https://github.com/woctezuma/discord-members-metadata

## Install packages

In [ ]:
%pip install --quiet transformers detoxify mediapy

## Download the image dataset

In [ ]:
%cd /content

for i in range(1, 3):
  fname = f"img_{i}.zip"

  !curl -OL https://github.com/woctezuma/discord-members-metadata/releases/download/img/{fname}
  !unzip -qq {fname}

## Download the text datasets

In [ ]:
%cd /content

!curl -OL https://github.com/woctezuma/discord-members-metadata/releases/download/bio/bios.json
!curl -OL https://github.com/woctezuma/discord-members-metadata/releases/download/metadata/members.json

## Define utils

In [ ]:
import json

from pathlib import Path

def save_to_json(data, fname):
  with Path(fname).open('w') as f:
    json.dump(data, f, indent=True)

def load_from_json(fname):
  with Path(fname).open() as f:
    data = json.load(f)
  return data

def safe_load_from_json(fname):
  try:
    data = load_from_json(fname)
  except FileNotFoundError:
    data = {}
  return data

In [ ]:
from pathlib import Path

def get_member_id(image_path):
  return Path(image_path).stem

In [ ]:
def get_output_fname(pipe, suffix = "", ext = ".json"):
  return pipe.model.name_or_path.replace('/', '_') + f'{suffix}{ext}'

In [ ]:
def save_detoxify_results(detoxify_results : dict, model_type: str = "", data_type: str =""):
  # This fixes:
  # > TypeError: Object of type float32 is not JSON serializable
  dd = {}
  for k,v in detoxify_results.items():
    dd[k] = {kk:float(vv) for kk,vv in v.items()}

  fname = f"detoxify_{model_type}_{data_type}.json"
  save_to_json(dd, fname)

## Classify images

Reference:
- https://github.com/woctezuma/stable-diffusion-safety-checker

### Preparation

Image dataset

In [ ]:
import functools
import os

import numpy as np

from pathlib import Path
from torchvision.datasets.folder import default_loader, is_image_file
from torchvision.transforms.functional import to_pil_image

@functools.lru_cache
def get_image_paths(path):
    paths = []
    for _dirpath, _dirnames, filenames in os.walk(path):
        paths.extend([str(Path(_dirpath) / filename) for filename in filenames])
    return sorted([fn for fn in paths if is_image_file(fn)])

class ImageFolder:

    def __init__(self, path, transform=None, loader=default_loader):
        self.samples = get_image_paths(path)
        self.loader = loader
        self.transform = transform

    def __getitem__(self, idx: int):
        assert 0 <= idx < len(self)
        img = self.loader(self.samples[idx])
        if self.transform:
            img = self.transform(img)
        return to_pil_image(img)

    def __len__(self):
        return len(self.samples)


Transform

In [ ]:
from torchvision import transforms

def get_target_image_size(resize_size=256, keep_ratio=True):
    return resize_size if keep_ratio else (resize_size, resize_size)

def get_transform(
    resize_size=256,
    keep_ratio=True,
    interpolation=transforms.InterpolationMode.BICUBIC,
):
    transforms_list = [
        transforms.Resize(
            get_target_image_size(resize_size, keep_ratio),
            interpolation=interpolation,
        ),
        transforms.ToTensor(),
    ]
    return transforms.Compose(transforms_list)

Data loader

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    """Collate function for data loader. Allows to have img of different size"""
    return batch

def get_dataloader(
    data_dir,
    transform = get_transform(),
    batch_size=8,
    collate_fn=collate_fn,
):
    dataset = ImageFolder(data_dir, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
    )
    return dataloader

### Run

Pipeline

In [ ]:
from transformers import pipeline

# https://huggingface.co/Falconsai/nsfw_image_detection
pipe = pipeline("image-classification",
                model="Falconsai/nsfw_image_detection",
                device="cuda")

Apply the workflow

In [ ]:
import torch

from tqdm.auto import tqdm

# For my use case, this cell required ~ 25 minutes.
data_path = "img/"
batch_size = 8

sample_fnames = []
safety_scores = []

loader = get_dataloader(data_path, batch_size = batch_size)

with torch.no_grad():
  for ii, images in enumerate(tqdm(loader)):
    out = pipe(images)

    sample_fnames += [
        loader.dataset.samples[ii * batch_size + jj]
        for jj in range(len(out))
    ]

    for dd in out:
      safety_scores += [ d["score"] for d in dd
                        if d["label"] == "normal" ]

### Save the results

Collate the IDs with the scores. At the same time, display the worst offenders.

In [ ]:
import torch

print(">>> Saving safety scores...")
fname = get_output_fname(pipe, suffix="_safety_scores", ext=".pth")
v = torch.asarray(safety_scores, dtype=torch.float16)
torch.save(v, fname)

In [ ]:
import json

print(">>> Saving image paths...")
fname = "img_list.json"
save_to_json(sample_fnames, fname)

In [ ]:
import mediapy as media

safety_score_threshold = 0.005
img_size = (128, 128)

for image_path, safety_score in sorted(
    zip(sample_fnames, safety_scores),
    key=lambda x: x[1]):
  member_id = get_member_id(image_path)

  if safety_score < safety_score_threshold:
    image = media.read_image(image_path)
    image = media.resize_image(image, img_size)

    print(f"{member_id} {safety_score:.2}")
    media.show_image(image)


## Check results obtained with Stable Diffusion Safety Checker

- https://github.com/woctezuma/stable-diffusion-safety-checker

In [ ]:
!curl -OL https://github.com/woctezuma/discord-members-metadata/releases/download/img/bad_concepts_scores.pth
!curl -OL https://github.com/woctezuma/discord-members-metadata/releases/download/img/img_list.json

In [ ]:
import torch

paths = load_from_json("img_list.json")

scores = torch.load("bad_concepts_scores.pth")
scores[scores<=0] = 0

In [ ]:
import mediapy as media

CONCEPT_ID_OF_INTEREST = 3
NUM_DISPLAYED_RESULTS = 25

DISPLAY_SIZE = (128, 128)

vec = scores[:,CONCEPT_ID_OF_INTEREST]
# To sum over all the concepts:
# vec = scores.sum(dim=1)

d = {k:v for k,v in zip(paths, vec)}

counter = 0

for k,v in sorted(d.items(), key=lambda x: x[1], reverse=True):
  print(f"{k} | score: {v:.2} for concept n°{CONCEPT_ID_OF_INTEREST}")
  media.show_image(media.resize_image(media.read_image(k), DISPLAY_SIZE))

  counter += 1

  if v <=0 or NUM_DISPLAYED_RESULTS < counter:
    break

## Classify texts

- https://huggingface.co/unitary/toxic-bert
- https://github.com/unitaryai/detoxify

In [ ]:
from detoxify import Detoxify

model_index = 2
model_types = ["original", "unbiased", "multilingual"]

model_type = model_types[model_index]
print(model_type)

text_model = Detoxify(model_type)

In [ ]:
bios_dict = load_from_json("bios.json")
start_from_scratch = False

if start_from_scratch:
  results = {}

counter = 0

for member, bio in bios_dict.items():
  if member not in results:
    results[member] = text_model.predict(bio)

  counter += 1

  if counter % 100 == 0:
    print(f'{counter}/{len(bios_dict)}')

save_detoxify_results(results, model_type, data_type = "bios")

In [ ]:
members_dict = load_from_json("members.json")
nickname_fields = ["display_name", "global_name", "name", "nick"]
start_from_scratch = False

if start_from_scratch:
  results_for_nicknames = {}

counter = 0

for member, metadata in members_dict.items():
  if member not in results_for_nicknames:

    all_the_nicknames = list(set([metadata[k] for k in nickname_fields if metadata[k]]))
    input_text = ' '.join(all_the_nicknames)

    results_for_nicknames[member] = text_model.predict(input_text)

  counter += 1

  if counter % 100 == 0:
    print(f'{counter}/{len(members_dict)}')